In [1]:
import os

In [3]:
import pandas as pd
test = pd.read_csv("../dataset/train.csv")

In [5]:
test['discourse_effectiveness'].astype('category').cat.codes.unique()

array([0, 2, 1], dtype=int8)

In [ ]:
import torch.cudaDB_PATH = f'../input/processed'
VOCAB_DIR = \
    os.path.join(DB_PATH, 'vocab')

In [3]:
vocab =  [line.split('\t')[0] for line in open(os.path.join(VOCAB_DIR,'spm.vocab'), encoding='utf-8').readlines()]
token2id = dict([(w,i) for i,w in enumerate(vocab)])

In [4]:

token2id

{'<unk>': 0,
 '<s>': 1,
 '</s>': 2,
 '▁t': 3,
 '▁th': 4,
 '▁a': 5,
 '▁the': 6,
 '▁s': 7,
 'in': 8,
 '▁w': 9,
 're': 10,
 '▁o': 11,
 '▁c': 12,
 '▁i': 13,
 'en': 14,
 'er': 15,
 '▁b': 16,
 'ou': 17,
 'le': 18,
 'at': 19,
 '▁p': 20,
 '▁to': 21,
 'on': 22,
 'ing': 23,
 '▁h': 24,
 'or': 25,
 '▁f': 26,
 '▁m': 27,
 'an': 28,
 '▁d': 29,
 '▁an': 30,
 'es': 31,
 '▁e': 32,
 '▁be': 33,
 '▁and': 34,
 'ar': 35,
 'ent': 36,
 '▁in': 37,
 'ot': 38,
 '▁of': 39,
 'll': 40,
 'om': 41,
 '▁n': 42,
 'us': 43,
 'it': 44,
 'ct': 45,
 '▁st': 46,
 'ed': 47,
 'as': 48,
 '▁g': 49,
 '▁that': 50,
 've': 51,
 'ld': 52,
 '▁is': 53,
 '▁y': 54,
 '▁wh': 55,
 '▁it': 56,
 'is': 57,
 'ch': 58,
 'al': 59,
 '▁l': 60,
 'ion': 61,
 '▁you': 62,
 'ould': 63,
 '▁v': 64,
 '▁on': 65,
 'dent': 66,
 'ic': 67,
 'ol': 68,
 '▁ha': 69,
 'ut': 70,
 '▁for': 71,
 'ay': 72,
 '▁they': 73,
 'et': 74,
 '▁re': 75,
 'ow': 76,
 '▁stu': 77,
 'ke': 78,
 'op': 79,
 '▁student': 80,
 'id': 81,
 'ir': 82,
 '▁are': 83,
 '▁have': 84,
 'ce': 85,
 'ro': 86,


In [5]:
import cate_dataset
import pandas as pd

In [6]:
class CFG:
    learning_rate = 3.0e-4 # 러닝 레이트
    batch_size=1024 # 배치 사이즈
    num_workers=4 # 워커의 개수
    print_freq=100 # 결과 출력 빈도
    start_epoch=0 # 시작 에폭
    num_train_epochs=10 # 학습할 에폭수
    warmup_steps=100 # lr을 서서히 증가시킬 step 수
    max_grad_norm=10 # 그래디언트
    weight_decay=0.01
    dropout=0.2 # dropout 확률
    hidden_size=512 # 은닉 크기
    intermediate_size = 256 # TRANSFORMER 셀의 intermediate 크기
    nlayers=2 # BERT 의 층수
    nheads=8 # BERT 의 head 개수
    seq_len=64 # 토큰의 최대 길이
    #n_b_cls = 57 + 1  # 대카테고리 개수
    n_b_cls = 3  # 대카테고리 개수
    # n_m_cls = 552 + 1  # 중카테고리 개수
    # n_s_cls = 3190 + 1  # 소카테고리 개수
    # n_d_cls = 404 + 1  # 세카테고리 개수
    vocab_size = 32000 # 토큰의 유니크 인덱스 개수
#    img_feat_size = 2048 #  이미지 피처 벡터의 크기
    type_vocab_size = 30 # 타입의 유니크 인덱스 개수
    csv_path = os.path.join(DB_PATH, 'train.csv')
#    h5_path = os.path.join(DB_PATH, 'train_img_feat.h5')

In [7]:
print('loading...')
train_df = pd.read_csv(CFG.csv_path , dtype={'tokens' : str})

loading...


In [8]:
import cate_dataset

In [9]:
train_db = cate_dataset.CateDataset(train_df, token2id,
                                CFG.seq_len , CFG.type_vocab_size)

In [10]:
from torch.utils.data import DataLoader


In [11]:
train_loader = DataLoader(
        train_db, batch_size=CFG.batch_size , shuffle=True , drop_last=True,
        num_workers=CFG.num_workers,pin_memory=True)


In [12]:
train_loader


In [13]:
import cate_model

In [14]:
# 카테고리 분류기 모델을 생성함
model = cate_model.CateClassifier(CFG)

In [15]:
model


CateClassifier(
  (text_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 512, padding_idx=0)
      (position_embeddings): Embedding(64, 512)
      (token_type_embeddings): Embedding(30, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_af

In [16]:
model.cuda()

CateClassifier(
  (text_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 512, padding_idx=0)
      (position_embeddings): Embedding(64, 512)
      (token_type_embeddings): Embedding(30, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_af

In [18]:

def count_parameter(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [19]:
print('parameters: ', count_parameter(model))
# 파라미터를 가중치라고도 한다

parameters:  19854339


In [22]:
import torch
# GPU가 2개 이상이면 데이터 페럴렐로 학습 가능하게 만듦
n_gpu = torch.cuda.device_count()
if n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [23]:
n_gpu

1

In [2]:

import tensorflow as tf

In [12]:
const = tf.constant([1])

In [10]:
import numpy as np
arr = [1]
const.split(1,1)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'split'

In [11]:
arr.split(1,1)

AttributeError: 'list' object has no attribute 'split'

In [7]:
type(const)

NameError: name 'const' is not defined

In [12]:
import torch.nn as nn

t = np.array([[[0, 1, 2],
               [3, 4, 5]],
              [[6, 7, 8],
               [9, 10, 11]]])
ft = torch.FloatTensor(t)

ModuleNotFoundError: No module named 'torch'

In [10]:
!pip install torchvision

In [11]:
!pip install torch

In [14]:
import torch



ModuleNotFoundError: No module named 'torch'